# 4.2.1 원-핫 인코딩

In [1]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요"

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


# 4.2.2 희소 표현과 분산 표현

# 4.2.3 Word2Vec

In [10]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # 헤더 제거
    return data

# 학습 시간 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time()-start)

# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time()-start)

# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print('3) Word2Vec 모델 학습 완료: ', time.time()-start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time()-start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print("corpus_count: ", model.corpus_count)
print("corpus_total_words: ", model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  0.5440161228179932
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  85.18547296524048
3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료:  104.20827007293701
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  104.28915309906006
corpus_count:  200000
corpus_total_words:  1076896


- sentences: Word2Vec 모델 학습에 필요한 문장 데이터. Word2Vec 모델의 입력값으로 사용됨
- vector_size: 단어 임베딩 벡터의 차원 (크기)
- window: 주변 단어 윈도우의 크기
- hs: 1(모델학습에 softmax 사용), 0(negative 옵션 값이 0이 아닐 때 음수 샘플링으로 사용)
- min_count: 단어 최소 빈도수 제한 (tjfwjdehls min_count 빈도수 이하의 단어들은 학습하지 않음)
- sg: 0(CBOW 모델), 1(skip-gram 모델)

In [12]:
# 위에서 저장한 모델 파일(nvmc.model) 활용 예제

from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words: ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑: ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

corpus_total_words:  1076896
사랑:  [-6.47868961e-02 -5.15041411e-01  1.84044376e-01 -2.70353258e-01
  2.10005179e-01 -7.28060752e-02  5.51567078e-02  1.71588123e-01
 -2.54164226e-02 -9.50366780e-02 -4.62177962e-01  2.48968467e-01
  6.77067637e-02 -1.15221396e-01 -1.67074159e-01  2.59793609e-01
 -1.54248610e-01  2.61751637e-02 -9.80559662e-02 -2.05659956e-01
  1.48594096e-01  1.40902817e-01 -2.01088369e-01  2.12129846e-01
 -2.77419090e-01  9.35066566e-02  2.02679023e-01 -1.95244282e-01
 -1.31147623e-01  2.09806591e-01  7.06647336e-02  3.23556550e-02
  1.14561051e-01 -1.87904418e-01  1.61628217e-01 -1.71363165e-04
  1.23991847e-01 -3.19565763e-03 -4.57614474e-02 -1.28738582e-01
 -1.18259050e-01  1.70185506e-01 -1.37026757e-01 -1.84761196e-01
  8.44365656e-02  1.96666703e-01 -3.47218752e-01  4.45370227e-02
  1.17285505e-01  5.21596372e-02  3.15069914e-01 -8.34720656e-02
 -1.53974906e-01 -2.46060677e-02  4.47047621e-01  1.56436771e-01
  2.77648658e-01 -2.12821007e-01  1.50535792e-01 -9.7211